## Import and setup

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
import torch
import numpy as np
import pandas as pd
import torch.optim as optim
from utils.preprocessing import *
from models.gsr_model import GSRNet
from utils.gsr_train import train, test
from models.ops import GraphUnet


## Define model and hyperparameters

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define model arguments
class Args:
    epochs = 200
    lr = 0.0001
    splits = 5
    lmbda = 16
    lr_dim = 160
    hr_dim = 268  # Adapted for dataset
    hidden_dim = 268
    padding = 0  # No padding

args = Args()

# Load dataset
subjects_adj, subjects_labels, test_adj = data()

# Cross-validation split
from sklearn.model_selection import KFold
cv = KFold(n_splits=args.splits, shuffle=True, random_state=42)

ks = [0.9, 0.7, 0.6, 0.5]  # Pooling rates
model = GSRNet(ks, args).to(device)
optimizer = optim.Adam(model.parameters(), lr=args.lr)


## Train and save model

In [ ]:
for train_index, test_index in cv.split(subjects_adj):
    train_adj, val_adj = subjects_adj[train_index], subjects_adj[test_index]
    train_labels, val_labels = subjects_labels[train_index], subjects_labels[test_index]

    train(model, optimizer, train_adj, train_labels, args)
    test(model, val_adj, val_labels, args)

# Save trained model
torch.save(model.state_dict(), "gsr_net_trained.pth")
print("Model saved!")


In [ ]:
# Load trained model
model.load_state_dict(torch.load("gsr_net_trained.pth"))
model.eval()  # Ensure evaluation mode

# Generate predictions
preds_list = []
for lr in test_adj:
    lr = torch.from_numpy(lr).float().to(device)
    with torch.no_grad():  # Disable gradient computation
        preds, _, _, _ = model(lr)  # Get predictions
    preds_list.append(preds.detach().cpu().numpy())  # Convert to NumPy

# Extract only upper triangular part (excluding diagonal)
def upper_triangular_flatten(matrix):
    return matrix[np.triu_indices(268, k=1)]  # k=1 excludes diagonal

# Apply upper triangular flattening to all predictions
preds_list = [upper_triangular_flatten(pred) for pred in preds_list]

# Create DataFrame
submission_df = pd.DataFrame({
    "ID": np.arange(1, len(np.concatenate(preds_list)) + 1),  # Sequential IDs
    "Predicted": np.concatenate(preds_list)  # Flattened predictions
})

# Save CSV
submission_df.to_csv("submission.csv", index=False)
print("Submission file created!")


## Submit to kaggle

In [ ]:
!kaggle competitions submit -c dgl-2025-brain-graph-super-resolution-challenge -f submission.csv -m "GSR-Net submission"